In [2]:
import SeasonData as sdata
import pandas as pd
import ipywidgets
import pprint

In [3]:
# we assume it is a regular season

season = '2022'
rs_data = sdata.NHLData()
rs_data.fetch_regular_season(year = season)
game_count = len(rs_data.regular_season[season])

def plot_game(game_number):    
    game_data = rs_data.regular_season['2022'][game_number-1]
    
    print(game_data['startTimeUTC'])
    print(f"Game ID: {game_number}; {game_data['homeTeam']['abbrev']} (home) vs {game_data['awayTeam']['abbrev']} (away)")
    
    col1 = ['', 'Teams', 'Goals', 'SoG']
    col2 = ["Home",f"{game_data['homeTeam']['abbrev']}", f"{game_data['homeTeam']['score']}", f"{game_data['homeTeam']['sog']}"]
    col3 = ["Away",f"{game_data['awayTeam']['abbrev']}", f"{game_data['awayTeam']['score']}", f"{game_data['awayTeam']['sog']}"]
    print('')
    for c1, c2, c3 in zip(col1, col2, col3):
        print(f'{c1:<18} {c2:<18} {c3:<18}')
    
    event_count = len(game_data['plays'])
    
    ipywidgets.interact(plot_game_event, event_number=(1, event_count,1), game_data=ipywidgets.fixed(game_data))

def plot_game_event(game_data, event_number):
    event_data = game_data['plays'][event_number-1]
    print("infos de l'evenement")
    pprint.pprint(event_data)

ipywidgets.interact(plot_game, game_number=(1, game_count,1))


Data imported: 0


interactive(children=(IntSlider(value=656, description='game_number', max=1312, min=1), Output()), _dom_classe…

<function __main__.plot_game(game_number)>